In [109]:
# import основынх библиотек
import pandas as pd
import numpy as np
# import с библиотеки sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# Убираем назойливые предупреждения
import warnings
warnings.filterwarnings("ignore")

In [110]:
# Считываем и выводим информацию о датасете
df = pd.read_csv("Titanic-Dataset.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Первые шаги по построению модели:

1. **Пропущенные значения:**
   - Посмотреть на количество пропущенных значений в каждом поле.
   - Решить, как обработать пропущенные значения (например, заполнение средним, медианой, удаление строк).

2. **Уникальные значения:**
   - Оценить количество уникальных значений в каждом поле.
   - Идентифицировать категориальные признаки с высоким количеством уникальных значений.

3. **Типы данных:**
   - Оценить типы данных в каждом поле (целочисленные, вещественные, категориальные).
   - Преобразовать типы данных при необходимости (например, категориальные данные в формат category).

### Замечание:

Эти шаги помогут подготовить данные к построению модели. Обработка пропущенных значений и работы с типами данных важны для эффективного обучения модели и предотвращения проблем при анализе данных.


In [111]:
def analyze_data(df):
    """
    Функция для анализа данных в DataFrame.

    Parameters:
    - df: pandas DataFrame

    Returns:
    - info_df: pandas DataFrame, содержащий информацию о количестве уникальных значений,
              пропущенных значениях и типах данных для каждого столбца в DataFrame.
    """
    # Рассчитываем количество уникальных значений
    unique_counts = pd.Series({col: df[col].nunique() for col in df.columns}, name='Уникальные значения')

    # Рассчитываем количество пропущенных значений
    missing_values = pd.Series(df.isnull().sum(), name='Пропущенные значения')

    # Получаем информацию о типах данных
    data_types = pd.Series(df.dtypes, name='Типы данных')

    # Объединяем созданные строки в один DataFrame
    info_df = pd.concat([unique_counts, missing_values, data_types], axis=1)

    return info_df

In [112]:
analyze_data(df)

,Уникальные значения,Пропущенные значения,Типы данных
PassengerId,891,0,int64
Survived,2,0,int64
Pclass,3,0,int64
Name,891,0,object
Sex,2,0,object
Age,88,177,float64
SibSp,7,0,int64
Parch,7,0,int64
Ticket,681,0,object
Fare,248,0,float64


### Обнаруженные проблемы:

1. **Пропущенные значения в поле "Age":** Существует значительное количество пропущенных значений в этом поле, что может повлиять на анализ данных и построение модели.

2. **Пропущенные значения в поле "Cabin":** В поле "Cabin" присутствует слишком много пропущенных значений, что делает его использование нецелесообразным для анализа и моделирования.

3. **Пропущенные значения в поле "Embarked":** Обнаружены два пропущенных значения в поле "Embarked", которые требуют внимания.

### Предложенные решения:

1. **Заполнение пропущенных значений в поле "Age":** Предлагаю рассчитать средний возраст и заменить пропущенные значения в поле "Age" этим средним возрастом. Это поможет сохранить данные в этом поле и избежать искажения общей картины.

2. **Исключение поля "Cabin":** Поскольку в поле "Cabin" слишком много пропущенных значений и анализ данных указывает на его незначительность для модели, рекомендуется полностью исключить это поле из дальнейшего рассмотрения.

3. **Обработка пропущенных значений в поле "Embarked":** Необходимо решить, как обработать два пропущенных значения в поле "Embarked".

### План действий:

- Выполнить расчет среднего возраста и заменить пропущенные значения в поле "Age".
- Исключить поле "Cabin" из датасета.
- Удалить пропущенные значения в поле "Embarked".

### Замечание:

Данный подход поможет обеспечить чистоту и надежность данных, что является ключевым элементом успешного построения модели и анализа данных.


In [113]:
mean_age = df['Age'].mean() # Рассчитываем средний возраст

df['Age'].fillna(mean_age, inplace=True) # Заполняем значения NaN в поле 'Age' средним возрастом

df.drop("Cabin", axis=1, inplace=True) # Удаляем поле 'Cabin'

df = df[df['Embarked'].notna()] # Фильтруем строки, где 'Embarked' не является NaN

### Повторный анализ данных:

In [114]:
analyze_data(df)

,Уникальные значения,Пропущенные значения,Типы данных
PassengerId,889,0,int64
Survived,2,0,int64
Pclass,3,0,int64
Name,889,0,object
Sex,2,0,object
Age,89,0,float64
SibSp,7,0,int64
Parch,7,0,int64
Ticket,680,0,object
Fare,247,0,float64



### Удаление уникальных полей:

1. **Поле "Name":** Уникальные идентификаторы пассажиров в поле "Name" не предоставляют значимой информации для обучения модели. Это поле скорее всего будет восприниматься как уникальный идентификатор, что может привести к избыточной сложности в модели.

2. **Поле "Ticket":** Уникальные номера билетов в поле "Ticket" также не предоставляют обобщенной информации, которая может быть полезной для модели. Включение этого поля может сделать модель менее интерпретируемой и менее способной к обобщению.

3. **Поле "PassengerId":** Этот уникальный идентификатор пассажира в поле "PassengerId" не несет смысловой нагрузки для обучения модели. Его использование в модели может привести к переобучению и ухудшению способности модели к обобщению.

### Замечание:

Исключение этих уникальных полей поможет упростить модель, уменьшить ее сложность и повысить ее способность к обобщению на новые данные.


In [115]:
df.drop(["Name", "Ticket", "PassengerId"], axis=1, inplace=True) # Удаление вышеперечисленных полей

### Преобразование категориальных признаков в числовые значения

Для эффективного обучения модели машинного обучения необходимо преобразовать категориальные признаки в числовые значения. Для этой цели используется LabelEncoder из библиотеки scikit-learn.


In [116]:
# Инициализируем LabelEncoder
label_encoder = LabelEncoder()
# Проходимся цыклом for по полям датфрейма и кодируем поля в числа
for column in df.select_dtypes(include=['object']).columns:
    df[column] = label_encoder.fit_transform(df[column])

In [117]:
analyze_data(df)

,Уникальные значения,Пропущенные значения,Типы данных
Survived,2,0,int64
Pclass,3,0,int64
Sex,2,0,int64
Age,89,0,float64
SibSp,7,0,int64
Parch,7,0,int64
Fare,247,0,float64
Embarked,3,0,int64


In [124]:
df.head() # Убидимся что все правильно

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


### Разделение данных на тренировочную и тестовую выборки

Для оценки производительности модели необходимо разделить данные на две части: тренировочную и тестовую выборки. Тренировочная выборка используется для обучения модели, а тестовая - для оценки ее способности обобщения на новые данные. Для этой цели используется train_test_split из библиотеки scikit-learn.

In [118]:
# Выделяем признаки (X) и целевую переменную (y)
x = df.drop("Survived", axis=1)  # x содержит все столбцы, кроме "Survived"
y = df["Survived"]  # y содержит только столбец "Survived"

# Разделяем данные на тренировочную и тестовую выборки (например, 75% - тренировочная, 25% - тестовая)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

# Теперь у вас есть X_train и y_train для обучения модели, и X_test и y_test для ее тестирования.

### Обучение модели с использованием RandomForestClassifier и GridSearchCV

Для построения модели и оптимизации ее параметров используем RandomForestClassifier из библиотеки scikit-learn, а также GridSearchCV для перебора параметров.

```python
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


In [119]:
# ВНИМАНИЕ! Этот код может выполняться очень долго и сильно нагрузить ваше устройство

# Создаем модель RandomForestClassifier
rf_clf = RandomForestClassifier(random_state=42)

# Определяем параметры, которые мы хотим проверить
params = {"n_estimators" : range(1, 161, 40), # Количество деревьев в лесу
          "max_depth" : range(1, 15),         # Максимальная глубина деревьев
          "min_samples_leaf" : range(1, 5),  # Минимальное количество примеров, необходимых для листа дерева
          "min_samples_split" : range(2, 5)} # Минимальное количество примеров, необходимых для разделения узла

# Ищем лучшие парамтеры молеи
searh = GridSearchCV(rf_clf, params, cv=5, n_jobs=-1)

# Обучение модели 
searh.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(1, 12),
                         'min_samples_leaf': range(1, 10),
                         'min_samples_split': range(2, 10),
                         'n_estimators': range(10, 201, 10)})

In [120]:
# Получаем лучшие параметры
best_params = searh.best_params_

# Получаем лучшую модель
best_model = searh.best_estimator_

In [121]:
best_params # Вывод лучших параметров подобранные GridSearchCV

{'max_depth': 11,
 'min_samples_leaf': 1,
 'min_samples_split': 4,
 'n_estimators': 160}

### Оценка результатов модели на тренировочной и тестовой выборках

Теперь, когда модель обучена, давайте оценим ее производительность на обеих выборках (тренировочной и тестовой) и сравним результаты.

In [122]:
(best_model.score(X_train, y_train), # Резултаты на тренировочной выборке
best_model.score(X_test, y_test)) # Резултаты на тестовой выборке

(0.933933933933934, 0.8071748878923767)